# 批量归⼀化
本节我们介绍批量归⼀化（batch normalization）层，它能让较深的神经⽹络的训练变得更加容
易 [1]。在[“实战Kaggle⽐赛：预测房价”](../3dlbasic/3.16kaggle-house-price.ipynb) ⼀节⾥，我们对输⼊数据做了标准化处理：处理后的任
意⼀个特征在数据集中所有样本上的均值为0、标准差为1。标准化处理输⼊数据使各个特征的分
布相近：这往往更容易训练出有效的模型。

通常来说，数据标准化预处理对于浅层模型就⾜够有效了。随着模型训练的进⾏，当每层中参数
更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经⽹络来说，即使输⼊数据已做标准
化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定
性通常令我们难以训练出有效的深度模型。

批量归⼀化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归⼀化利⽤小批量上
的均值和标准差，不断调整神经⽹络中间输出，从而使整个神经⽹络在各层的中间输出的数值更
稳定。批量归⼀化和下⼀节将要介绍的残差⽹络为训练和设计深度模型提供了两类重要思路。
## 批量归⼀化层
对全连接层和卷积层做批量归⼀化的⽅法稍有不同。下⾯我们将分别介绍这两种情况下的批量归⼀化。
### 对全连接层做批量归⼀化
我们先考虑如何对全连接层做批量归⼀化。通常，我们将批量归⼀化层置于全连接层中的仿射变
换和激活函数之间。设全连接层的输⼊为u，权重参数和偏差参数分别为W和b，激活函数为ϕ。
设批量归⼀化的运算符为BN。那么，使⽤批量归⼀化的全连接层的输出为 $\phi(BN(X))$
其中批量归⼀化输⼊x由仿射变换 $x=Wu+b$,考虑一个由$m$个样本组成的小批量，仿射变换的输出
为一个新的小批量$\mathcal{B} = \{\boldsymbol{x}^{(1)}, \ldots, \boldsymbol{x}^{(m)} \}$。
它们正是批量归一化层的输入。对于小批量$\mathcal{B}$中任意样本$\boldsymbol{x}^{(i)} \in \mathbb{R}^d, 1 \leq  i \leq m$，批量归一化层的输出同样是$d$维向量

$y^{(i)} = \text{BN}(x^{(i)})$

并由以下⼏步求得。⾸先，对小批量B求均值和⽅差：

$$\mu_B \leftarrow \frac 1 m \sum_{i=1}^m x^{(i)}$$

$$\sigma_B^2 \leftarrow \frac 1 m \sum_{i=1}^m(\mu_B - x^{(i)})^2$$,

其中的平⽅计算是按元素求平⽅。接下来，使⽤按元素开⽅和按元素除法对$x^{(i)}$标准化：

$\hat x^{(i)} \leftarrow \frac{x^{(i)} - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$

这⾥ϵ > 0是⼀个很小的常数，保证分⺟⼤于0。在上⾯标准化的基础上，批量归⼀化层引⼊了两
个可以学习的模型参数，拉伸（scale）参数 $\gamma$ 和偏移（shift）参数 $\beta$。这两个参数和$x^{(i)}$形状相
同，皆为d维向量。它们与$\hat x^{(i)}$分别做按元素乘法（符号$\odot$）和加法计算：

$\hat y^{(i)} \leftarrow \gamma \odot \hat x^{(i)} + \beta$

⾄此，我们得到了$x^{(i)}$的批量归⼀化的输出$y^{(i)}$。值得注意的是，可学习的拉伸和偏移参数保留了
不对$x^{(i)}$做批量归⼀化的可能：此时只需学出$\gamma = \sqrt{\sigma_B^2 + \epsilon}和\beta = \mu_B$
我们可以对此这样理解：如果批量归⼀化⽆益，理论上，学出的模型可以不使⽤批量归⼀化。
### 对卷积层做批量归⼀化
对卷积层来说，批量归⼀化发⽣在卷积计算之后、应⽤激活函数之前。如果卷积计算输出多个通
道，我们需要对这些通道的输出分别做批量归⼀化，且每个通道都拥有独⽴的拉伸和偏移参数，
并均为标量。设小批量中有m个样本。在单个通道上，假设卷积计算输出的⾼和宽分别为p和q。我
们需要对该通道中m × p × q个元素同时做批量归⼀化。对这些元素做标准化计算时，我们使⽤相
同的均值和⽅差，即该通道中m × p × q个元素的均值和⽅差。
### 预测时的批量归⼀化
使⽤批量归⼀化训练时，我们可以将批量⼤小设得⼤⼀点，从而使批量内样本的均值和⽅差的计
算都较为准确。将训练好的模型⽤于预测时，我们希望模型对于任意输⼊都有确定的输出。因此，
单个样本的输出不应取决于批量归⼀化所需要的随机小批量中的均值和⽅差。⼀种常⽤的⽅法是
通过移动平均估算整个训练数据集的样本均值和⽅差，并在预测时使⽤它们得到确定的输出。可
⻅，和丢弃层⼀样，批量归⼀化层在训练模式和预测模式下的计算结果也是不⼀样的。
## 从零开始实现
下⾯我们通过NDArray来实现批量归⼀化层

In [ ]:
import d2lzh as d2l
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import nn
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
	# 通过autograd来判断当前模式是训练模式还是预测模式
	if not autograd.is_training():
		# 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
		X_hat = (X - moving_mean) / nd.sqrt(moving_var + eps)
	else:
		assert len(X.shape) in (2, 4)
		if len(X.shape) == 2:
			# 使用全连接层的情况，计算特征维上的均值和方差
			mean = X.mean(axis=0)
			var = ((X - mean) ** 2).mean(axis=0)
		else:
			# 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
			# X的形状以便后面可以做广播运算
			mean = X.mean(axis=(0, 2, 3), keepdims=True)
			var = ((X - mean) ** 2).mean(axis=(0, 2, 3), keepdims=True)
		# 训练模式下用当前的均值和方差做标准化
		X_hat = (X - mean) / nd.sqrt(var + eps)
		# 更新移动平均的均值和方差
		moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
		moving_var = momentum * moving_var + (1.0 - momentum) * var
	Y = gamma * X_hat + beta  # 拉伸和偏移
	return Y, moving_mean, moving_var

接下来，我们⾃定义⼀个BatchNorm层。它保存参与求梯度和迭代的拉伸参数gamma和偏移参
数beta，同时也维护移动平均得到的均值和⽅差，以便能够在模型预测时被使⽤。 BatchNorm实
例所需指定的num_features参数对于全连接层来说应为输出个数，对于卷积层来说则为输出
通道数。该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4

In [ ]:
class BatchNorm(nn.Block):
	def __init__(self, num_features, num_dims, **kwargs):
		super(BatchNorm, self).__init__(**kwargs)
		if num_dims == 2:
			shape = (1, num_features)
		else:
			shape = (1, num_features, 1, 1)
		# 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
		self.gamma = self.params.get('gamma', shape=shape, init=init.One())
		self.beta = self.params.get('beta', shape=shape, init=init.Zero())
		# 不参与求梯度和迭代的变量，全在内存上初始化成0
		self.moving_mean = nd.zeros(shape)
		self.moving_var = nd.zeros(shape)

	def forward(self, X):
		# 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
		if self.moving_mean.context != X.context:
			self.moving_mean = self.moving_mean.copyto(X.context)
			self.moving_var = self.moving_var.copyto(X.context)
		# 保存更新过的moving_mean和moving_var
		Y, self.moving_mean, self.moving_var = batch_norm(
			X, self.gamma.data(), self.beta.data(), self.moving_mean,
			self.moving_var, eps=1e-5, momentum=0.9)
		return Y

### 使⽤批量归⼀化层的LeNet
下⾯我们修改[“卷积神经⽹络（LeNet）”](5.5lenet.ipynb) 这⼀节介绍的LeNet模型，从而应⽤批量归⼀化层。我
们在所有的卷积层或全连接层之后、激活层之前加⼊批量归⼀化层。

In [ ]:
net = nn.Sequential()
net.add(nn.Conv2D(6, kernel_size=5),
        BatchNorm(6, num_dims=4),
        nn.Activation('sigmoid'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(16, kernel_size=5),
        BatchNorm(16, num_dims=4),
        nn.Activation('sigmoid'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Dense(120),
        BatchNorm(120, num_dims=2),
        nn.Activation('sigmoid'),
        nn.Dense(84),
        BatchNorm(84, num_dims=2),
        nn.Activation('sigmoid'),
        nn.Dense(10))

下⾯我们训练修改后的模型。

In [ ]:
lr, num_epochs, batch_size, ctx = 1.0, 5, 256, d2l.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

最后我们查看第⼀个批量归⼀化层学习到的拉伸参数gamma和偏移参数beta。
net[1].gamma.data().reshape((-1,)), net[1].beta.data().reshape((-1,))

## 简洁实现
与我们刚刚⾃⼰定义的BatchNorm类相⽐， Gluon中nn模块定义的BatchNorm类使⽤起来更
加简单。它不需要指定⾃⼰定义的BatchNorm类中所需的num_features和num_dims参数值。
在Gluon中，这些参数值都将通过延后初始化而⾃动获取。下⾯我们⽤Gluon实现使⽤批量归⼀
化的LeNet。